In [2]:
import pandas as pd
from datetime import datetime
import data_processing

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [10]:
raw_data = pd.read_excel("../data/raw_data.xlsx")

In [14]:
raw_data.columns.to_list()

['TalentID',
 'Stützpunktname',
 'Koordinatorengebiet',
 'MasterID',
 'Geschlecht',
 'Geburtstag',
 'Geburtsjahr',
 'T1subj_TestID',
 'T1subj_Test_RangeID',
 'T1subj_Stützpunktname',
 'T1subj_Altersklasse',
 'T1subj_Geschlecht',
 'T1subj_Geburtstag',
 'T1subj_Testdatum',
 'T1subj_StützpunktID',
 'T1subj_Spielertyp',
 'T1subj_Technik_Dom_Fuss',
 'T1subj_Technik_Nicht_Dom_Fuss',
 'T1subj_Kopfballtechnik',
 'T1subj_Kond_Fähigkeiten',
 'T1subj_Taktik_offensiv_vor',
 'T1subj_Taktik_offensiv_während',
 'T1subj_Taktik_offensiv_nach',
 'T1subj_Taktik_defensiv_vor',
 'T1subj_Taktik_defensiv_während',
 'T1subj_Taktik_defensiv_nach',
 'T1subj_Spielintelligenz',
 'T1subj_Technik_Reichweite',
 'T1subj_Falltechniken',
 'T1subj_Technik_Flanken',
 'T1subj_Technik_Eröff_Forts',
 'T1subj_Taktik_Stellungssspiel',
 'T1subj_Taktik_1vs1',
 'T1subj_Psy_Motivation',
 'T1subj_Psy_Volition',
 'T1subj_Psy_Sozial',
 'T1subj_Aktuelle_Leistungsfähigkeit',
 'T1subj_Zukünftiges_Leistungsniveau',
 'T1subj_Bemerkungen'

# Cleaning and treatment

In [11]:
# Apply cleaning functions
clean_df = data_processing.fill_gender(raw_data)
clean_df = data_processing.fill_birth_dates(clean_df)
clean_df = data_processing.fill_koordinatorengebiet(clean_df)
clean_df = data_processing.fill_stuetzpunktname(clean_df)

# Filter out columns
cols = ['TalentID', 'Stützpunktname', 'Koordinatorengebiet', 'Geschlecht', 'Geburtstag', 'Geburtsjahr',
 'T1subj_Altersklasse', 'T1subj_Spielertyp','T1subj_Technik_Dom_Fuss','T1subj_Technik_Nicht_Dom_Fuss','T1subj_Kopfballtechnik','T1subj_Kond_Fähigkeiten',
 'T1subj_Taktik_offensiv_vor','T1subj_Taktik_offensiv_während','T1subj_Taktik_offensiv_nach','T1subj_Taktik_defensiv_vor',
 'T1subj_Taktik_defensiv_während','T1subj_Taktik_defensiv_nach','T1subj_Spielintelligenz','T1subj_Technik_Reichweite',
 'T1subj_Falltechniken','T1subj_Technik_Flanken','T1subj_Technik_Eröff_Forts','T1subj_Taktik_Stellungssspiel','T1subj_Taktik_1vs1',
 'T1subj_Psy_Motivation','T1subj_Psy_Volition','T1subj_Psy_Sozial','T1subj_Aktuelle_Leistungsfähigkeit','T1subj_Zukünftiges_Leistungsniveau',
 'T2subj_Altersklasse', 'T2subj_Spielertyp','T2subj_Technik_Dom_Fuss','T2subj_Technik_Nicht_Dom_Fuss','T2subj_Kopfballtechnik','T2subj_Kond_Fähigkeiten',
 'T2subj_Taktik_offensiv_vor','T2subj_Taktik_offensiv_während','T2subj_Taktik_offensiv_nach','T2subj_Taktik_defensiv_vor',
 'T2subj_Taktik_defensiv_während','T2subj_Taktik_defensiv_nach','T2subj_Spielintelligenz','T2subj_Technik_Reichweite',
 'T2subj_Falltechniken','T2subj_Technik_Flanken','T2subj_Technik_Eröff_Forts','T2subj_Taktik_Stellungssspiel',
 'T2subj_Taktik_1vs1','T2subj_Psy_Motivation','T2subj_Psy_Volition','T2subj_Psy_Sozial',
 'T2subj_Aktuelle_Leistungsfähigkeit','T2subj_Zukünftiges_Leistungsniveau','T2subj_Bemerkungen', 
 'T3subj_Altersklasse','T3subj_Spielertyp', 'T3subj_Technik_Dom_Fuss', 'T3subj_Technik_Nicht_Dom_Fuss', 'T3subj_Kopfballtechnik',
 'T3subj_Kond_Fähigkeiten', 'T3subj_Taktik_offensiv_vor', 'T3subj_Taktik_offensiv_während', 'T3subj_Taktik_offensiv_nach',
 'T3subj_Taktik_defensiv_vor', 'T3subj_Taktik_defensiv_während', 'T3subj_Taktik_defensiv_nach', 'T3subj_Spielintelligenz',
 'T3subj_Technik_Reichweite', 'T3subj_Falltechniken', 'T3subj_Technik_Flanken', 'T3subj_Technik_Eröff_Forts',
 'T3subj_Taktik_Stellungssspiel', 'T3subj_Taktik_1vs1', 'T3subj_Psy_Motivation', 'T3subj_Psy_Volition', 'T3subj_Psy_Sozial',
 'T3subj_Aktuelle_Leistungsfähigkeit', 'T3subj_Zukünftiges_Leistungsniveau','T3subj_Bemerkungen',
 'T25_Grösse', 'T25_Gewicht', 'T25_Altersklasse','T25_SL10', 'T25_SL20', 'T25_GW', 'T25_DR', 'T25_BK', 'T25_BJ', 'T25_TS', 'T25_SC',
 'T27_Grösse', 'T27_Gewicht', 'T27_Altersklasse','T27_SL10', 'T27_SL20', 'T27_GW', 'T27_DR', 'T27_BK', 'T27_BJ', 'T27_TS', 'T27_SC',
 'T29_Grösse', 'T29_Gewicht', 'T29_Altersklasse', 'T29_SL10','T29_SL20','T29_GW', 'T29_DR', 'T29_BK', 'T29_BJ', 'T29_TS', 'T29_SC',
 'U12_FR_SL10',	'U13_FR_SL10', 'U14_FR_SL10', 'U15_FR_SL10',
 'U12_FR_SL20', 'U13_FR_SL20', 'U14_FR_SL20', 'U15_FR_SL20',
 'U12_FR_GW', 'U13_FR_GW', 'U14_FR_GW', 'U15_FR_GW',
 'U12_FR_DR','U13_FR_DR', 'U14_FR_DR', 'U15_FR_DR',
 'U12_FR_BK', 'U13_FR_BK', 'U14_FR_BK', 'U15_FR_BK',
 'U12_FR_TS', 'U13_FR_TS', 'U14_FR_TS'	,'U15_FR_TS',
 'U12_FR_BJ','U13_FR_BJ', 'U14_FR_BJ', 'U15_FR_BJ',	
 'U12_FR_SC', 'U13_FR_SC', 'U14_FR_SC', 'U15_FR_SC',	
 'U12_FR_Grösse', 'U13_FR_Grösse', 'U14_FR_Grösse', 'U15_FR_Grösse',
 'U12_FR_Gewicht','U13_FR_Gewicht','U14_FR_Gewicht', 'U15_FR_Gewicht'
]

clean_df = clean_df[cols].copy()



# Define mapping from original column names to English names.
# Edit this dict to include translations for your columns, e.g. 'AlterName': 'Age'
col_rename = {
    'Stützpunktname' : 'BaseName',
    'Koordinatorengebiet': 'CoordinationArea',
    'Geschlecht': 'Gender',
    'Geburtstag' : 'Birth',
    'Geburtsjahr': 'BirthYear',
    'T1subj_Altersklasse': 'T1subj_AK',
    'T2subj_Altersklasse': 'T2subj_AK',
    'T3subj_Altersklasse': 'T3subj_AK',
    'T25_Altersklasse': 'T25_AK',
    'T27_Altersklasse': 'T27_AK',
    'T29_Altersklasse': 'T29_AK',
}
# # Apply renaming (only keys present in col_rename will be changed)
clean_df = clean_df.rename(columns=col_rename)

# Create future_success column: take first non-null value from the three assessments
clean_df['future_success'] = clean_df[['T1subj_Zukünftiges_Leistungsniveau', 
                                        'T2subj_Zukünftiges_Leistungsniveau', 
                                        'T3subj_Zukünftiges_Leistungsniveau']].bfill(axis=1).iloc[:, 0]

# Create LZ binary target variable: 0 if future_success < 3, 1 if >= 3
clean_df['LZ'] = (clean_df['future_success'] >= 3).astype(int)

# Create relative_age: day of birth within the year (1-365/366)
clean_df['relative_age'] = clean_df['Birth'].apply(data_processing.calculate_relative_age)

# Save cleaned dataframe to CSV
clean_df.to_csv('../data/cleaned_data.csv', index=False)
clean_df.head()

'Geschlecht' missing after filling: 0
'Geburtstag' missing after filling: 0
'Geburtsjahre' missing after filling: 0
'Koordinatorengebiet' missing after filling: 1045
'Stützpunktname' missing after filling: 0


,TalentID,BaseName,CoordinationArea,Gender,Birth,BirthYear,T1subj_AK,T1subj_Spielertyp,T1subj_Technik_Dom_Fuss,T1subj_Technik_Nicht_Dom_Fuss,T1subj_Kopfballtechnik,T1subj_Kond_Fähigkeiten,T1subj_Taktik_offensiv_vor,T1subj_Taktik_offensiv_während,T1subj_Taktik_offensiv_nach,T1subj_Taktik_defensiv_vor,T1subj_Taktik_defensiv_während,T1subj_Taktik_defensiv_nach,T1subj_Spielintelligenz,T1subj_Technik_Reichweite,T1subj_Falltechniken,T1subj_Technik_Flanken,T1subj_Technik_Eröff_Forts,T1subj_Taktik_Stellungssspiel,T1subj_Taktik_1vs1,T1subj_Psy_Motivation,T1subj_Psy_Volition,T1subj_Psy_Sozial,T1subj_Aktuelle_Leistungsfähigkeit,T1subj_Zukünftiges_Leistungsniveau,T2subj_AK,T2subj_Spielertyp,T2subj_Technik_Dom_Fuss,T2subj_Technik_Nicht_Dom_Fuss,T2subj_Kopfballtechnik,T2subj_Kond_Fähigkeiten,T2subj_Taktik_offensiv_vor,T2subj_Taktik_offensiv_während,T2subj_Taktik_offensiv_nach,T2subj_Taktik_defensiv_vor,T2subj_Taktik_defensiv_während,T2subj_Taktik_defensiv_nach,T2subj_Spielintelligenz,T2subj_Technik_Reichweite,T2subj_Falltechniken,T2subj_Technik_Flanken,T2subj_Technik_Eröff_Forts,T2subj_Taktik_Stellungssspiel,T2subj_Taktik_1vs1,T2subj_Psy_Motivation,T2subj_Psy_Volition,T2subj_Psy_Sozial,T2subj_Aktuelle_Leistungsfähigkeit,T2subj_Zukünftiges_Leistungsniveau,T2subj_Bemerkungen,T3subj_AK,T3subj_Spielertyp,T3subj_Technik_Dom_Fuss,T3subj_Technik_Nicht_Dom_Fuss,T3subj_Kopfballtechnik,T3subj_Kond_Fähigkeiten,T3subj_Taktik_offensiv_vor,T3subj_Taktik_offensiv_während,T3subj_Taktik_offensiv_nach,T3subj_Taktik_defensiv_vor,T3subj_Taktik_defensiv_während,T3subj_Taktik_defensiv_nach,T3subj_Spielintelligenz,T3subj_Technik_Reichweite,T3subj_Falltechniken,T3subj_Technik_Flanken,T3subj_Technik_Eröff_Forts,T3subj_Taktik_Stellungssspiel,T3subj_Taktik_1vs1,T3subj_Psy_Motivation,T3subj_Psy_Volition,T3subj_Psy_Sozial,T3subj_Aktuelle_Leistungsfähigkeit,T3subj_Zukünftiges_Leistungsniveau,T3subj_Bemerkungen,T25_Grösse,T25_Gewicht,T25_AK,T25_SL10,T25_SL20,T25_GW,T25_DR,T25_BK,T25_BJ,T25_TS,T25_SC,T27_Grösse,T27_Gewicht,T27_AK,T27_SL10,T27_SL20,T27_GW,T27_DR,T27_BK,T27_BJ,T27_TS,T27_SC,T29_Grösse,T29_Gewicht,T29_AK,T29_SL10,T29_SL20,T29_GW,T29_DR,T29_BK,T29_BJ,T29_TS,T29_SC,U12_FR_SL10,U13_FR_SL10,U14_FR_SL10,U15_FR_SL10,U12_FR_SL20,U13_FR_SL20,U14_FR_SL20,U15_FR_SL20,U12_FR_GW,U13_FR_GW,U14_FR_GW,U15_FR_GW,U12_FR_DR,U13_FR_DR,U14_FR_DR,U15_FR_DR,U12_FR_BK,U13_FR_BK,U14_FR_BK,U15_FR_BK,U12_FR_TS,U13_FR_TS,U14_FR_TS,U15_FR_TS,U12_FR_BJ,U13_FR_BJ,U14_FR_BJ,U15_FR_BJ,U12_FR_SC,U13_FR_SC,U14_FR_SC,U15_FR_SC,U12_FR_Grösse,U13_FR_Grösse,U14_FR_Grösse,U15_FR_Grösse,U12_FR_Gewicht,U13_FR_Gewicht,U14_FR_Gewicht,U15_FR_Gewicht,future_success,LZ,relative_age
0,01765VFKTO000000VV0AG812VUQRIF2C,Memmingen,Bayern 4,0.0,1999-02-01,1999.0,U17,Feldspieler,3.0,2.0,3.0,2.0,2.0,3.0,2.0,3.0,3.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,187.0,78.1,U17,1.73,2.91,8.24,9.98,10.97,5.0,NaN,106.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.96,1.84,1.81,3.44,3.42,3.28,3.10,7.92,8.12,8.08,7.81,10.07,10.64,11.18,10.64,9.98,8.90,9.38,8.68,19.0,10.0,14.0,14.0,1.0,0.0,1.0,1.0,102.81,102.07,103.07,104.55,155.0,168.0,177.0,183.0,41.0,53.0,60.0,69.6,2.0,0,32
1,017666J46S000000VV0AG812VUQRIF2C,Memmingen,Bayern 4,0.0,1999-04-15,1999.0,U17,Feldspieler,3.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,188.0,80.8,U17,1.91,3.23,8.47,10.10,9.43,19.0,NaN,106.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.10,1.92,1.89,3.34,3.58,3.42,3.30,8.03,7.93,7.90,7.91,10.06,9.73,10.01,11

In [7]:
clean_data = pd.read_csv("../data/cleaned_data.csv")
clean_data.head()

C:\Users\eduar\AppData\Local\Temp\ipykernel_37684\4284412390.py:1: DtypeWarning: Columns (6,7,30,31,54,82,93) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_data = pd.read_csv("../data/cleaned_data.csv")


,TalentID,BaseName,CoordinationArea,Gender,Birth,BirthYear,T1subj_AK,T1subj_Spielertyp,T1subj_Technik_Dom_Fuss,T1subj_Technik_Nicht_Dom_Fuss,T1subj_Kopfballtechnik,T1subj_Kond_Fähigkeiten,T1subj_Taktik_offensiv_vor,T1subj_Taktik_offensiv_während,T1subj_Taktik_offensiv_nach,T1subj_Taktik_defensiv_vor,T1subj_Taktik_defensiv_während,T1subj_Taktik_defensiv_nach,T1subj_Spielintelligenz,T1subj_Technik_Reichweite,T1subj_Falltechniken,T1subj_Technik_Flanken,T1subj_Technik_Eröff_Forts,T1subj_Taktik_Stellungssspiel,T1subj_Taktik_1vs1,T1subj_Psy_Motivation,T1subj_Psy_Volition,T1subj_Psy_Sozial,T1subj_Aktuelle_Leistungsfähigkeit,T1subj_Zukünftiges_Leistungsniveau,T2subj_AK,T2subj_Spielertyp,T2subj_Technik_Dom_Fuss,T2subj_Technik_Nicht_Dom_Fuss,T2subj_Kopfballtechnik,T2subj_Kond_Fähigkeiten,T2subj_Taktik_offensiv_vor,T2subj_Taktik_offensiv_während,T2subj_Taktik_offensiv_nach,T2subj_Taktik_defensiv_vor,T2subj_Taktik_defensiv_während,T2subj_Taktik_defensiv_nach,T2subj_Spielintelligenz,T2subj_Technik_Reichweite,T2subj_Falltechniken,T2subj_Technik_Flanken,T2subj_Technik_Eröff_Forts,T2subj_Taktik_Stellungssspiel,T2subj_Taktik_1vs1,T2subj_Psy_Motivation,T2subj_Psy_Volition,T2subj_Psy_Sozial,T2subj_Aktuelle_Leistungsfähigkeit,T2subj_Zukünftiges_Leistungsniveau,T2subj_Bemerkungen,T3subj_AK,T3subj_Spielertyp,T3subj_Technik_Dom_Fuss,T3subj_Technik_Nicht_Dom_Fuss,T3subj_Kopfballtechnik,T3subj_Kond_Fähigkeiten,T3subj_Taktik_offensiv_vor,T3subj_Taktik_offensiv_während,T3subj_Taktik_offensiv_nach,T3subj_Taktik_defensiv_vor,T3subj_Taktik_defensiv_während,T3subj_Taktik_defensiv_nach,T3subj_Spielintelligenz,T3subj_Technik_Reichweite,T3subj_Falltechniken,T3subj_Technik_Flanken,T3subj_Technik_Eröff_Forts,T3subj_Taktik_Stellungssspiel,T3subj_Taktik_1vs1,T3subj_Psy_Motivation,T3subj_Psy_Volition,T3subj_Psy_Sozial,T3subj_Aktuelle_Leistungsfähigkeit,T3subj_Zukünftiges_Leistungsniveau,T3subj_Bemerkungen,T25_Grösse,T25_Gewicht,T25_AK,T25_SL10,T25_SL20,T25_GW,T25_DR,T25_BK,T25_BJ,T25_TS,T25_SC,T27_Grösse,T27_Gewicht,T27_AK,T27_SL10,T27_SL20,T27_GW,T27_DR,T27_BK,T27_BJ,T27_TS,T27_SC,T29_Grösse,T29_Gewicht,T29_AK,T29_SL10,T29_SL20,T29_GW,T29_DR,T29_BK,T29_BJ,T29_TS,T29_SC,U12_FR_SL10,U13_FR_SL10,U14_FR_SL10,U15_FR_SL10,U12_FR_SL20,U13_FR_SL20,U14_FR_SL20,U15_FR_SL20,U12_FR_GW,U13_FR_GW,U14_FR_GW,U15_FR_GW,U12_FR_DR,U13_FR_DR,U14_FR_DR,U15_FR_DR,U12_FR_BK,U13_FR_BK,U14_FR_BK,U15_FR_BK,U12_FR_TS,U13_FR_TS,U14_FR_TS,U15_FR_TS,U12_FR_BJ,U13_FR_BJ,U14_FR_BJ,U15_FR_BJ,U12_FR_SC,U13_FR_SC,U14_FR_SC,U15_FR_SC,U12_FR_Grösse,U13_FR_Grösse,U14_FR_Grösse,U15_FR_Grösse,U12_FR_Gewicht,U13_FR_Gewicht,U14_FR_Gewicht,U15_FR_Gewicht,future_success,LZ,relative_age
0,01765VFKTO000000VV0AG812VUQRIF2C,Memmingen,Bayern 4,0.0,1999-02-01,1999.0,U17,Feldspieler,3.0,2.0,3.0,2.0,2.0,3.0,2.0,3.0,3.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,187.0,78.1,U17,1.73,2.91,8.24,9.98,10.97,5.0,NaN,106.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.96,1.84,1.81,3.44,3.42,3.28,3.10,7.92,8.12,8.08,7.81,10.07,10.64,11.18,10.64,9.98,8.90,9.38,8.68,19.0,10.0,14.0,14.0,1.0,0.0,1.0,1.0,102.81,102.07,103.07,104.55,155.0,168.0,177.0,183.0,41.0,53.0,60.0,69.6,2.0,0,32
1,017666J46S000000VV0AG812VUQRIF2C,Memmingen,Bayern 4,0.0,1999-04-15,1999.0,U17,Feldspieler,3.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,188.0,80.8,U17,1.91,3.23,8.47,10.10,9.43,19.0,NaN,106.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.10,1.92,1.89,3.34,3.58,3.42,3.30,8.03,7.93,7.90,7.91,10.06,9.73,10.01,11

In [8]:
clean_data['LZ'].value_counts()

LZ
0    15261
1    14993
Name: count, dtype: int64

### Duplicated rows

In [12]:
#duplicated IDs - Goalkeeper and field player

clean_data['TalentID'].value_counts()[:21]

TalentID
01PIDSGBB4000001VV0AG813VVVK9KPP    2
01M5QRITMS000001VV0AG812VSF3IDCV    2
01PKEHK8V0000001VV0AG812VUNEJRST    2
01PQB9QNGO000001VV0AG813VS8416I1    2
01Q5R047EO000001VV0AG813VS93NFOQ    2
01PTNP2SI8000001VV0AG812VTTMBQN1    2
01QUOTLTHK000001VS54898EVV8NEGFI    2
01SFSOBOHK000001VS54898DVSP70I3G    2
01TBL1RCP4000001VS54898EVVLNNI37    2
01TBL6ST80000001VS54898EVVLNNI37    2
01TG05AANO000001VS54898EVUCU6B7O    2
01TMICI678000001VS54898EVTCISHSL    2
01TNM0OU5O000001VS54898EVTCISHSL    2
01U21CR0A4000001VS54898DVT7P3BG2    2
01UVNVR4NK000000VS54898EVV13EOL7    2
01V3L02130000001VS54898DVT0U52DJ    2
020JGD091O000001VS54898EVSIREAE3    2
020S4C48EO000000VS54898EVUOJ8HQ7    2
020VALOP8C000001VS54898DVV861QDV    2
0210U6PU90000001VS54898DVV861QDV    2
021BS21LQG000000VS54898EVV96NMP4    2
Name: count, dtype: int64

# Split data

## Split by Gender

In [9]:
# # Split by gender

men_df = clean_data[clean_data["Gender"] == 0]
women_df = clean_data[clean_data["Gender"] == 1]

# Save gender dataframes to CSV
men_df.to_csv('../data/male_data.csv', index=False)
women_df.to_csv('../data/female_data.csv', index=False)

## Split by AK

In [7]:
# Split by AK

# Base columns (use renamed English names for core fields)
cols_base = [
    'TalentID', 'BaseName', 'CoordinationArea', 'Gender', 'Birth', 'BirthYear','relative_age',
    'T1subj_AK','T2subj_AK','T3subj_AK', 'T25_AK', 'T27_AK','T29_AK',
    'T1subj_Spielertyp','T2subj_Spielertyp','T3subj_Spielertyp',
    'T1subj_Technik_Dom_Fuss','T1subj_Technik_Nicht_Dom_Fuss','T1subj_Kopfballtechnik',
    'T1subj_Kond_Fähigkeiten','T1subj_Taktik_offensiv_vor','T1subj_Taktik_offensiv_während','T1subj_Taktik_offensiv_nach',
    'T1subj_Taktik_defensiv_vor', 'T1subj_Taktik_defensiv_während','T1subj_Taktik_defensiv_nach','T1subj_Spielintelligenz',
    'T1subj_Psy_Motivation','T1subj_Psy_Volition','T1subj_Psy_Sozial',
    'T2subj_Technik_Dom_Fuss', 'T3subj_Technik_Nicht_Dom_Fuss', 'T3subj_Kopfballtechnik',
    'T2subj_Kond_Fähigkeiten', 'T2subj_Taktik_offensiv_vor', 'T2subj_Taktik_offensiv_während', 'T2subj_Taktik_offensiv_nach',
    'T2subj_Taktik_defensiv_vor', 'T2subj_Taktik_defensiv_während', 'T2subj_Taktik_defensiv_nach', 'T2subj_Spielintelligenz',
    'T2subj_Psy_Motivation', 'T2subj_Psy_Volition', 'T2subj_Psy_Sozial',
    'T3subj_Technik_Dom_Fuss', 'T3subj_Technik_Nicht_Dom_Fuss', 'T3subj_Kopfballtechnik',
    'T3subj_Kond_Fähigkeiten', 'T3subj_Taktik_offensiv_vor', 'T3subj_Taktik_offensiv_während', 'T3subj_Taktik_offensiv_nach',
    'T3subj_Taktik_defensiv_vor', 'T3subj_Taktik_defensiv_während', 'T3subj_Taktik_defensiv_nach', 'T3subj_Spielintelligenz',
    'T3subj_Psy_Motivation', 'T3subj_Psy_Volition', 'T3subj_Psy_Sozial',
    'T1subj_Zukünftiges_Leistungsniveau', 'T2subj_Zukünftiges_Leistungsniveau', 'T3subj_Zukünftiges_Leistungsniveau',
    'future_success', 'LZ'
]

male_data = pd.read_csv("../data/male_data.csv")

# Create subsets for each age group
u12_df = data_processing.create_ak_subset(male_data, 'U12', cols_base)
u13_df = data_processing.create_ak_subset(male_data, 'U13', cols_base)
u14_df = data_processing.create_ak_subset(male_data, 'U14', cols_base)
u15_df = data_processing.create_ak_subset(male_data, 'U15', cols_base)

# Preview
u12_df.head()

C:\Users\eduar\AppData\Local\Temp\ipykernel_12316\4219685108.py:24: DtypeWarning: Columns (6,7,54,82,93) have mixed types. Specify dtype option on import or set low_memory=False.
  male_data = pd.read_csv("../data/male_data.csv")


U12 subset created: 10392 rows
U13 subset created: 8822 rows
U14 subset created: 5498 rows
U15 subset created: 3106 rows


,TalentID,BaseName,CoordinationArea,Gender,Birth,BirthYear,relative_age,T1subj_AK,T2subj_AK,T3subj_AK,T25_AK,T27_AK,T29_AK,T1subj_Spielertyp,T2subj_Spielertyp,T3subj_Spielertyp,T1subj_Technik_Dom_Fuss,T1subj_Technik_Nicht_Dom_Fuss,T1subj_Kopfballtechnik,T1subj_Kond_Fähigkeiten,T1subj_Taktik_offensiv_vor,T1subj_Taktik_offensiv_während,T1subj_Taktik_offensiv_nach,T1subj_Taktik_defensiv_vor,T1subj_Taktik_defensiv_während,T1subj_Taktik_defensiv_nach,T1subj_Spielintelligenz,T1subj_Psy_Motivation,T1subj_Psy_Volition,T1subj_Psy_Sozial,T2subj_Technik_Dom_Fuss,T3subj_Technik_Nicht_Dom_Fuss,T3subj_Kopfballtechnik,T2subj_Kond_Fähigkeiten,T2subj_Taktik_offensiv_vor,T2subj_Taktik_offensiv_während,T2subj_Taktik_offensiv_nach,T2subj_Taktik_defensiv_vor,T2subj_Taktik_defensiv_während,T2subj_Taktik_defensiv_nach,T2subj_Spielintelligenz,T2subj_Psy_Motivation,T2subj_Psy_Volition,T2subj_Psy_Sozial,T3subj_Technik_Dom_Fuss,T3subj_Technik_Nicht_Dom_Fuss,T3subj_Kopfballtechnik,T3subj_Kond_Fähigkeiten,T3subj_Taktik_offensiv_vor,T3subj_Taktik_offensiv_während,T3subj_Taktik_offensiv_nach,T3subj_Taktik_defensiv_vor,T3subj_Taktik_defensiv_während,T3subj_Taktik_defensiv_nach,T3subj_Spielintelligenz,T3subj_Psy_Motivation,T3subj_Psy_Volition,T3subj_Psy_Sozial,T1subj_Zukünftiges_Leistungsniveau,T2subj_Zukünftiges_Leistungsniveau,T3subj_Zukünftiges_Leistungsniveau,future_success,LZ,U12_FR_SL10,U12_FR_SL20,U12_FR_GW,U12_FR_DR,U12_FR_BK,U12_FR_BJ,U12_FR_SC,U12_FR_Grösse,U12_FR_Gewicht
696,01EGRDAON4000000VV0AG813VV4DPKDH,Greifswald,Mecklenburg-Vorpommern,0.0,2004-02-25,2004.0,56,U12,U13,NaN,U12,U13,NaN,Feldspieler,Feldspieler,NaN,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,NaN,NaN,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,2.0,0,2.17,3.76,8.31,9.99,8.33,3.0,102.78,145.0,NaN
1139,01F5BS5IOC000000VV0AG812VT9160M2,Neubukow,Mecklenburg-Vorpommern,0.0,2004-09-10,2004.0,254,U12,U13,NaN,U12,NaN,NaN,Feldspieler,Feldspieler,NaN,3.0,2.0,2.0,3.0,2.0,3.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,NaN,NaN,3.0,3.0,3.0,3.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,0,2.02,3.49,8.11,10.91,8.50,2.0,103.07,139.0,NaN
1169,01F9LPABC0000000VV0AG813VSNGFIOA,Greiz,Thüringen,0.0,2004-04-07,2004.0,98,U12,NaN,NaN,U12,NaN,NaN,Feldspieler,NaN,NaN,2.0,1.0,0.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,0,1.84,3.36,8.37,10.97,11.35,1.0,102.08,159.0,NaN
1312,01GD8TE6I8000000VV0AG812VSUNUM0H,Glauchau,Sachsen,0.0,2004-10-12,2004.0,286,U12,NaN,NaN,U12,NaN,NaN,Feldspieler,NaN,NaN,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,1,2.10,3.63,7.84,10.37,9.58,2.0,102.36,145.0,NaN
1441,01HHD84O1O000000VV0AG812VUK0C6FA,Löwenberg,Brandenburg,0.0,2004-02-28,2004.0,59,U12,NaN,NaN,U12,NaN,NaN,Feldspieler,NaN,NaN,3.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0,1.99,3.45,7.81,9.58,8.87,8.0,105.36,142.0,NaN


In [12]:
u15_df[(u15_df['T25_AK'] == 'U15')][['U15_FR_SL10','U15_FR_SL20','U15_FR_GW','U15_FR_DR','U15_FR_BK','U15_FR_BJ','U15_FR_SC','U15_FR_Grösse','U15_FR_Gewicht']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 919 entries, 0 to 2670
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   U15_FR_SL10     877 non-null    float64
 1   U15_FR_SL20     909 non-null    float64
 2   U15_FR_GW       915 non-null    float64
 3   U15_FR_DR       909 non-null    float64
 4   U15_FR_BK       903 non-null    float64
 5   U15_FR_BJ       916 non-null    float64
 6   U15_FR_SC       889 non-null    float64
 7   U15_FR_Grösse   898 non-null    float64
 8   U15_FR_Gewicht  898 non-null    float64
dtypes: float64(9)
memory usage: 71.8 KB


## Overall Male

In [8]:
u12_df = pd.read_csv("../data/u12_data_male.csv")
u13_df = pd.read_csv("../data/u13_data_male.csv")
u14_df = pd.read_csv("../data/u14_data_male.csv")
u15_df = pd.read_csv("../data/u15_data_male.csv")

C:\Users\eduar\AppData\Local\Temp\ipykernel_12316\2629264930.py:1: DtypeWarning: Columns (7,8,10,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  u12_df = pd.read_csv("../data/u12_data_male.csv")
C:\Users\eduar\AppData\Local\Temp\ipykernel_12316\2629264930.py:2: DtypeWarning: Columns (7,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  u13_df = pd.read_csv("../data/u13_data_male.csv")


In [10]:
#Refine AK subsets for modeling

refined_u12 = data_processing.refine_ak_dataset(u12_df, 'U12')
refined_u13 = data_processing.refine_ak_dataset(u13_df, 'U13') 
refined_u14 = data_processing.refine_ak_dataset(u14_df, 'U14') 
refined_u15 = data_processing.refine_ak_dataset(u15_df, 'U15') 

refined_u12.head()

,TalentID,AK,relative_age,height,weight,SL20,GW,DR,BK,BJ,SKSC_TAK,SKSC_TEC,SKSC_KON,SKSC_PSY,LZ
0,01EGRDAON4000000VV0AG813VV4DPKDH,U12,56,145.0,35.540552,3.76,8.31,9.99,8.33,3.0,1.857143,1.333333,2.0,1.666667,0
1,01F5BS5IOC000000VV0AG812VT9160M2,U12,254,139.0,35.540552,3.49,8.11,10.91,8.50,2.0,2.428571,2.333333,3.0,2.000000,0
2,01F9LPABC0000000VV0AG813VSNGFIOA,U12,98,159.0,35.540552,3.36,8.37,10.97,11.35,1.0,1.428571,1.000000,2.0,2.000000,0
3,01GD8TE6I8000000VV0AG812VSUNUM0H,U12,286,145.0,35.540552,3.63,7.84,10.37,9.58,2.0,0.714286,1.333333,2.0,2.000000,1
4,01HHD84O1O000000VV0AG812VUK0C6FA,U12,59,142.0,35.540552,3.45,7.81,9.58,8.87,8.0,2.714286,2.333333,3.0,2.666667,0


In [13]:
merged = data_processing.merge_refined_ak_datasets()
merged.head()

Loaded 10392 rows from U12
Loaded 8822 rows from U13
Loaded 5498 rows from U14
Loaded 3106 rows from U15

Total rows before deduplication: 27818
Total rows after deduplication: 19177

Distribution by AK:
AK
U12    10392
U13     4580
U14     2687
U15     1518
Name: count, dtype: int64

Saved to ../data/merged_male.csv


,TalentID,AK,relative_age,height,weight,SL20,GW,DR,BK,BJ,SKSC_TAK,SKSC_TEC,SKSC_KON,SKSC_PSY,LZ
24712,01A3QVST7O000000VV0AG812VTLC714F,U15,62,175.0,50.0,3.48,8.49,10.30,7.99,6.0,2.000000,1.333333,2.0,2.000000,1
24713,01AALE9DE4000000VV0AG813VV8TC69P,U15,207,170.0,51.0,3.41,7.85,10.54,8.02,23.0,1.000000,1.000000,1.0,1.000000,1
24714,01AIC7GS84000000VV0AG812VSS1EAOO,U15,162,168.0,63.0,3.08,7.58,9.47,8.02,9.0,2.857143,2.333333,2.0,2.333333,0
24715,01AM5S25I4000000VV0AG813VTJJDC6I,U15,14,180.0,67.0,3.12,7.51,8.81,7.75,15.0,1.285714,1.666667,1.0,1.000000,1
24716,01AURR6P2C000000VV0AG812VUDCV8V9,U15,26,177.0,67.0,3.25,7.80,9.48,8.11,16.0,2.000000,1.666667,3.0,2.000000,1


## Overall Female

In [14]:
female_data = pd.read_csv("../data/female_data.csv")

# Create subsets for each age group
u12_df_female = data_processing.create_ak_subset(female_data, 'U12', cols_base, gender='female')
u13_df_female = data_processing.create_ak_subset(female_data, 'U13', cols_base, gender='female')
u14_df_female = data_processing.create_ak_subset(female_data, 'U14', cols_base, gender='female')
u15_df_female = data_processing.create_ak_subset(female_data, 'U15', cols_base, gender='female')

U12 subset created: 700 rows
U13 subset created: 673 rows
U14 subset created: 479 rows
U15 subset created: 295 rows


In [15]:
u12_df_female = pd.read_csv("../data/u12_data_female.csv")
u13_df_female = pd.read_csv("../data/u13_data_female.csv")
u14_df_female = pd.read_csv("../data/u14_data_female.csv")
u15_df_female = pd.read_csv("../data/u15_data_female.csv")

In [16]:
#Refine AK subsets for modeling
refined_u12_female = data_processing.refine_ak_dataset(u12_df_female, 'U12', 'female')
refined_u13_female = data_processing.refine_ak_dataset(u13_df_female, 'U13', 'female') 
refined_u14_female = data_processing.refine_ak_dataset(u14_df_female, 'U14', 'female') 
refined_u15_female = data_processing.refine_ak_dataset(u15_df_female, 'U15', 'female') 

refined_u12_female.head()

,TalentID,AK,relative_age,height,weight,SL20,GW,DR,BK,BJ,SKSC_TAK,SKSC_TEC,SKSC_KON,SKSC_PSY,LZ
0,01I30RNUSK000001VV0AG813VU8QTUB0,U12,222,145.0,35.276,3.78,8.20,10.51,9.66,16.0,2.285714,2.333333,2.0,2.333333,0
1,01IBEPDDEC000000VV0AG812VTTMS6V3,U12,82,148.0,35.276,3.81,8.28,11.15,9.82,1.0,3.000000,2.666667,3.0,3.000000,0
2,01IQJ20F5G000001VV0AG812VT15LRCC,U12,29,144.0,35.276,3.67,8.18,10.93,10.22,3.0,1.142857,1.666667,2.0,1.333333,0
3,01JMIKR9E4000000VV0AG812VVLC3CII,U12,286,145.0,35.276,3.69,8.14,9.53,10.00,6.0,2.285714,2.000000,3.0,3.000000,0
4,01JVHPLN84000000VV0AG812VSUK31BF,U12,19,139.0,35.276,3.96,8.36,11.14,10.09,1.0,0.714286,0.666667,1.0,1.000000,1


In [17]:
merged_female = data_processing.merge_refined_ak_datasets(gender='female')
merged_female.head()

Loaded 700 rows from U12
Loaded 673 rows from U13
Loaded 479 rows from U14
Loaded 295 rows from U15

Total rows before deduplication: 2147
Total rows after deduplication: 1527

Distribution by AK:
AK
U12    700
U13    397
U14    270
U15    160
Name: count, dtype: int64

Saved to ../data/merged_female.csv


,TalentID,AK,relative_age,height,weight,SL20,GW,DR,BK,BJ,SKSC_TAK,SKSC_TEC,SKSC_KON,SKSC_PSY,LZ
1373,01DPKIJV6C000000VV0AG813VUERA7KH,U14,87,153.000000,49.000000,3.43,8.04,11.70,9.89,5.0,1.571429,1.333333,1.0,1.333333,1
1852,01E68E2VI4000000VV0AG813VUNPP5IQ,U15,228,176.000000,56.000000,3.74,8.28,11.67,12.18,1.0,2.000000,2.000000,3.0,2.000000,1
1853,01E7GP60OC000000VV0AG813VUNPP5IQ,U15,247,154.000000,40.000000,3.66,8.09,10.34,9.24,18.0,2.571429,2.000000,3.0,3.000000,0
1854,01E7H5R9G0000000VV0AG812VUQ9IPJS,U15,162,172.000000,60.000000,3.54,9.20,11.50,7.96,13.0,2.571429,2.333333,3.0,2.666667,0
1855,01E7J367S0000000VV0AG813VUNPP5IQ,U15,109,164.429577,53.556338,3.66,8.26,11.32,8.11,7.0,2.714286,2.333333,3.0,3.000000,0
